In [ ]:
"""
Created on Wed Jan 15 09:28:16 2020
Group 7
@authors : G.H.
"""

In [74]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread
import numpy as np
import pandas as pd
import math
import operator
import os
from PIL import Image
from tqdm import tqdm_notebook as tqdm
import random
import pickle

import shutil
from shutil import copyfile

# keras : librairie de deep learning
import keras
from keras.models import Sequential, Model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, InputLayer, ReLU, AveragePooling2D
from keras.optimizers import SGD
from keras.utils import np_utils
from keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

# import for transfer learning
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

# Lecure avec keras

In [76]:
# Reading of csv file SEATGURU annontate
csv_file = pd.read_csv('../g7_SEATGURU_annotate.csv', sep=';')
Airbus_file = csv_file.loc[csv_file['aircraft_manufacturer']  == 'Airbus',:]
Airbus_file = Airbus_file.loc[Airbus_file['view'] == 'Int']
#Boeing_file = csv_file.loc[csv_file['aircraft_manufacturer']  == 'Boeing',:]
#Boeing_file = Boeing_file.loc[Boeing_file['view'] == 'Int']
# List name pictures
list_img_Airbus = list(Airbus_file['name'])

A320_file = Airbus_file.loc[Airbus_file['aircraft_type'] == 'A320']
list_img_A320 = list(A320_file['name'])
A321_file = Airbus_file.loc[Airbus_file['aircraft_type'] == 'A321']
list_img_A321 = list(A321_file['name'])
A330_file = Airbus_file.loc[Airbus_file['aircraft_type'] == 'A330']
list_img_A330 = list(A330_file['name'])
A350_file = Airbus_file.loc[Airbus_file['aircraft_type'] == 'A350']
list_img_A350 = list(A350_file['name'])
A380_file = Airbus_file.loc[Airbus_file['aircraft_type'] == 'A380']
list_img_A380 = list(A380_file['name'])

#list_img_Boeing = list(Boeing_file['name'])

In [77]:
list_img_A380

['Etihad_Airways_Airbus_A380_0.jpg',
 'Qantas_Airways_Airbus_A380_B_4.jpg',
 'Qantas_Airways_Airbus_A380_B_1.jpg',
 'Emirates_Airlines_Airbus_A380_Two_Class_2.jpg',
 'Air_France_Airbus_A380_V2_0.jpg',
 'Malaysia_Airlines_Airbus_A380_new_0.jpg',
 'Etihad_Airways_Airbus_A380_2.jpg',
 'Korean_Air_Airbus_380-800_V2_0.jpg',
 'British_Airways_Airbus_A380_new_2.jpg',
 'Emirates_Airlines_Airbus_A380_B_1.jpg',
 'Air_France_Airbus_A380_V2_1.jpg',
 'British_Airways_Airbus_A380_new_3.jpg',
 'Emirates_Airlines_Airbus_A380_B_2.jpg',
 'Lufthansa_Airbus_A380_V3_4.jpg',
 'Emirates_Airlines_Airbus_A380_Two_Class_1.jpg',
 'Emirates_Airlines_Airbus_A380_0.jpg',
 'Thai_Airways_Airbus_A380_new_2.jpg',
 'Emirates_Airlines_Airbus_A380_Two_Class_4.jpg',
 'Malaysia_Airlines_Airbus_A380_new_2.jpg',
 'Thai_Airways_Airbus_A380_new_4.jpg',
 'Etihad_Airways_Airbus_A380_3.jpg',
 'Thai_Airways_Airbus_A380_new_0.jpg',
 'Qantas_Airways_Airbus_A380_B_2.jpg',
 'Asiana_Airbus_A380_3.jpg',
 'Asiana_Airbus_A380_0.jpg',
 'Emi

In [78]:
path_seatguru = './../All Data/ANALYSE IMAGE/IMG SEATGURU/'
path_hackathon = './../Inputs Hackathon/'
new_path_train = './../Split_data/Hack_Seatguru/Int/data_train'
new_path_val = './../Split_data/Hack_Seatguru/Int/data_val'
new_paths = [new_path_train, new_path_val]

In [79]:
# function to split data in train and test samples for Seatguru pictures
def split_train_test_Seatguru_cat(new_paths, path, airbus_planes = [list_img_A320, list_img_A321, list_img_A330,
                                                                 list_img_A350, list_img_A380], 
                                  split_limit = .7, s = 8):
  

    for plane in airbus_planes:

        if plane == list_img_A320:
            classe = 'A320'
        if plane == list_img_A321:
            classe = 'A321'
        if plane == list_img_A330:
            classe = 'A330'
        if plane == list_img_A350:
            classe = 'A350'
        if plane == list_img_A380:
            classe = 'A380'
        
        shutil.rmtree(new_paths[0] + '/' + classe, ignore_errors = True)
        os.makedirs(new_paths[0] + '/' + classe, exist_ok = True)
        shutil.rmtree(new_paths[1] + '/' + classe, ignore_errors = True)
        os.makedirs(new_paths[1]+ '/' + classe, exist_ok = True)
        
        random.seed(a = s)
        random.shuffle(plane)

        for pict in plane[:int(split_limit*len(plane))]:
            copyfile(path + '/' + pict, new_paths[0] + '/' + classe + '/' +  pict)
        for pict in plane[int(split_limit*len(plane)):]:
            copyfile(path + '/' + pict, new_paths[1] + '/' + classe + '/' +  pict)
        

In [80]:
# function to split data in train and test samples for Hackathon pictures
def  split_train_test_Hack_cat (new_paths:list, path:str, airbus_planes:list, split_limit:float=.7):
    
    for plane in airbus_planes:
        
        os.makedirs(new_paths[0] + '/' + plane, exist_ok = True)
        os.makedirs(new_paths[1] + '/' + plane, exist_ok = True)
        
        for pict in os.listdir(path + plane):
            rand = np.random.random()
            if rand <= split_limit:
                copyfile(path + plane + '/' + pict, new_paths[0] + '/' + plane + '/' + pict)
            else:
                copyfile(path + plane + '/' + pict, new_paths[1] + '/' + plane + '/' + pict)
      

In [81]:
# Creation of train and test samples for Seatguru pictures
split_train_test_Seatguru_cat(new_paths, path_seatguru)

In [82]:
# Creation of train and test samples for Hackathon pictures
split_train_test_Hack_cat(new_paths, path_hackathon, airbus_planes = ['A320', 'A330', 'A350', 'A380'])

In [83]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies


train_generator = train_datagen.flow_from_directory(new_path_train,
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_generator = test_datagen.flow_from_directory(new_path_val,
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 1237 images belonging to 5 classes.
Found 540 images belonging to 5 classes.


# Transfer learning

In [86]:
# create the base pre-trained model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# add a global spatial average pooling layer
x = base_model.output


x = Conv2D(256, kernel_size=(3, 3))(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

x = Conv2D(256, kernel_size=(3, 3))(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

#x = MaxPooling2D(pool_size=(2, 2))(x)

# vector
x = Flatten()(x)

# let's add a fully-connected layer
x = Dense(1024)(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

x = Dense(512)(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

x = Dense(256)(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)


# last layer which predict the class
predictions = Dense(5, activation = 'softmax')(x)

# this is the model we will trainll
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [ ]:
step_size_train=train_generator.n//train_generator.batch_size

# Define learning rate reducer
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', 
                              patience=2, 
                              verbose=1, 
                              factor=0.5, 
                              min_lr=0.00001)

# How to include it in your training
model.fit_generator(generator=train_generator,
                    steps_per_epoch=step_size_train,
                    epochs=5,
                    validation_data=test_generator,
                    callbacks=[reduce_lr])  # here

Epoch 1/5
38/38 [==============================] - 270s 7s/step - loss: 1.4962 - accuracy: 0.4174 - val_loss: 9.4158 - val_accuracy: 0.2019
Epoch 2/5
38/38 [==============================] - 276s 7s/step - loss: 0.4882 - accuracy: 0.8340 - val_loss: 2.3194 - val_accuracy: 0.3611
Epoch 3/5
24/38 [=================>............] - ETA: 1:16 - loss: 0.2163 - accuracy: 0.9297

In [72]:
def save_model_classes(path_mod, mod_name, train_generator, model): 
    shutil.rmtree(path_mod + mod_name, ignore_errors = True)
    os.makedirs(path_mod + mod_name)
    label_map = (train_generator.class_indices)
    with open(path_mod + mod_name + '/' + 'model_' + mod_name + '.pkl', "wb") as f:
        pickle.dump(label_map, f)
    model.save(path_mod + mod_name + '/' + 'model_'+ mod_name + '.h5')

In [75]:
save_model_classes("./../Models/", 'int_Airbus_Hack_Seatguru', train_generator, model)